takeout-2020...zipを解答。unzipだとファイルの文字コードでエラーになるので、unarchiverを使う

In [14]:
import json

In [7]:
with open("Takeout/ロケーション履歴/ロケーション履歴.json") as f:
    history = json.load(f)

In [9]:
list(history.keys())

['locations']

In [13]:
history["locations"][:3]

[{'timestampMs': '1373458943759',
  'latitudeE7': 357028875,
  'longitudeE7': 1395619739,
  'accuracy': 2868},
 {'timestampMs': '1373459484329',
  'latitudeE7': 357010784,
  'longitudeE7': 1394978473,
  'accuracy': 2614},
 {'timestampMs': '1373459665257',
  'latitudeE7': 357005741,
  'longitudeE7': 1394816046,
  'accuracy': 2820}]

In [17]:
with open("Takeout/ロケーション履歴/Semantic Location History/2020/2020_APRIL.json") as f:
    semantic = json.load(f)

In [20]:
list(semantic.keys())

['timelineObjects']

In [29]:
semantic["timelineObjects"][0]

{'activitySegment': {'startLocation': {'latitudeE7': 356804606,
   'longitudeE7': 1394478496},
  'endLocation': {'latitudeE7': 356863064, 'longitudeE7': 1394404190},
  'duration': {'startTimestampMs': '1586071496000',
   'endTimestampMs': '1586072665738'},
  'distance': 1052,
  'activityType': 'WALKING',
  'confidence': 'HIGH',
  'activities': [{'activityType': 'WALKING', 'probability': 88.5364830493927},
   {'activityType': 'STILL', 'probability': 3.532659634947777},
   {'activityType': 'CYCLING', 'probability': 3.1043093651533127},
   {'activityType': 'IN_PASSENGER_VEHICLE', 'probability': 3.059180825948715},
   {'activityType': 'IN_TRAIN', 'probability': 0.6859975401312113},
   {'activityType': 'RUNNING', 'probability': 0.6100533995777369},
   {'activityType': 'MOTORCYCLING', 'probability': 0.17618112033233047},
   {'activityType': 'IN_BUS', 'probability': 0.13319664867594838},
   {'activityType': 'IN_FERRY', 'probability': 0.09249935392290354},
   {'activityType': 'IN_SUBWAY', 'pro

In [24]:
len(history["locations"]), len(semantic["timelineObjects"])

(383250, 59)

semanticの方は2020/04だけだった・・。しかし、4月の1ヶ月で59点しかないということは、hitoryと1対1対応する訳ではなさそう

In [61]:
import datetime
import pandas as pd

locations = pd.DataFrame.from_dict(history["locations"])
locations["datetime"] = locations.timestampMs.map(lambda x: datetime.datetime.fromtimestamp(int(x)/1000))
locations2004 = locations.loc[[x.year == 2020 and x.month == 4 for x in locations.datetime]]
locations2004

,timestampMs,latitudeE7,longitudeE7,accuracy,activity,velocity,altitude,verticalAccuracy,heading,datetime
311798,1585709096732,356802685,1394477687,14,"[{'timestampMs': '1585709098664', 'activity': ...",NaN,110.0,2.0,NaN,2020-04-01 11:44:56.732
311799,1585709228087,356802411,1394477524,13,NaN,NaN,110.0,3.0,NaN,2020-04-01 11:47:08.087
311800,1585709348727,356802681,1394477654,14,NaN,NaN,110.0,3.0,NaN,2020-04-01 11:49:08.727
311801,1585709471557,356803138,1394478053,25,"[{'timestampMs': '1585709431853', 'activity': ...",NaN,109.0,3.0,NaN,2020-04-01 11:51:11.557
311802,1585709591698,356803101,1394477866,17,"[{'timestampMs': '1585709594492', 'activity': ...",NaN,110.0,3.0,NaN,2020-04-01 11:53:11.698
...,...,...,...,...,...,...,...,...,...,...
319508,1588256969902,356802459,1394477699,13,NaN,NaN,110.0,3.0,NaN,2020-04-30 23:29:29.902
319509,1588257329097,356802459,1394477699,13,NaN,NaN,110.0,3.0,NaN,2020-04-30 23:35:29.097
319510,1588257818095,356802459,1394477699,13,NaN,NaN,110.0,3.0,NaN,2020-04-30 23:43:38.095
319511,1588258221112,356802459,1394477699,13,NaN,NaN,110.0,3.0,NaN,2020-04-30 23:50:21.112


In [63]:
len(locations2004), locations2004.activity.count()

(7715, 2150)

In [65]:
locations2004.activity.iloc[0]

[{'timestampMs': '1585709098664',
  'activity': [{'type': 'STILL', 'confidence': 100}]}]

In [78]:
import pyproj

epsg_from, epsg_to = 4326, 6677
tr = pyproj.Transformer.from_proj(epsg_from, epsg_to)
xys = [tr.transform(lat/10**7, lng/10**7) for lat, lng in zip(locations.latitudeE7, locations.longitudeE7)]    
locations[["x", "y"]] = xys
locations

,timestampMs,latitudeE7,longitudeE7,accuracy,activity,velocity,altitude,verticalAccuracy,heading,datetime,x,y
0,1373458943759,357028875,1395619739,2868,NaN,NaN,NaN,NaN,NaN,2013-07-10 21:22:23.759,-32929.257007,-24555.785773
1,1373459484329,357010784,1394978473,2614,NaN,NaN,NaN,NaN,NaN,2013-07-10 21:31:24.329,-33112.029868,-30359.418153
2,1373459665257,357005741,1394816046,2820,NaN,NaN,NaN,NaN,NaN,2013-07-10 21:34:25.257,-33162.834373,-31829.488621
3,1373459845824,357003463,1394763803,2753,NaN,NaN,NaN,NaN,NaN,2013-07-10 21:37:25.824,-33186.400987,-32302.352467
4,1373459966250,357005075,1394638697,2609,NaN,NaN,NaN,NaN,NaN,2013-07-10 21:39:26.250,-33164.328855,-33434.432947
...,...,...,...,...,...,...,...,...,...,...,...,...
383245,1607044343720,355853153,1397253045,87,NaN,NaN,41.0,2.0,NaN,2020-12-04 10:12:23.720,-46001.436758,-9790.038276
383246,1607044464083,355853153,1397253045,1653,"[{'timestampMs': '1607044346522', 'activity': ...",NaN,41.0,2.0,NaN,2020-12-04 10:14:24.083,-46001.436758,-9790.038276
383247,1607044585132,356141415,1397360164,1083,"[{'timestampMs': '1607044588597', 'activity': ...",NaN,NaN,NaN,NaN,2020-12-04 10:16:25.132,-42804.454420,-8816.117473
383248,1607044709718,356274644,1397390657,110,NaN,NaN,NaN,NaN,NaN,2020-12-04 10:18:29.718,-41326.668308,-8538.459440
